In [1]:
import tensorflow as tf

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/talipov/MyProjects/projectX/acosta_env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import numpy as np

In [3]:
MAX_LENGTH = 100
EMBEDDING_SIZE = 100

In [5]:
x = tf.placeholder(
    shape =[None, MAX_LENGTH, EMBEDDING_SIZE],
    dtype=tf.float32
)

In [6]:
x

<tf.Tensor 'Placeholder:0' shape=(?, 100, 100) dtype=float32>

In [75]:
_x= tf.reshape(x, shape=(-1, 1, MAX_LENGTH, EMBEDDING_SIZE))

In [76]:
_x

<tf.Tensor 'Reshape_7:0' shape=(?, 1, 100, 100) dtype=float32>

In [15]:
tf.layers.conv2d(inputs=_x, filters=32, kernel_size=(1,2), strides=1, padding="VALID")

<tf.Tensor 'conv2d_6/BiasAdd:0' shape=(?, 1, 99, 32) dtype=float32>

In [16]:
tf.layers.conv2d(inputs=_x, filters=32, kernel_size=(1,3), strides=1, padding="VALID")

<tf.Tensor 'conv2d_7/BiasAdd:0' shape=(?, 1, 98, 32) dtype=float32>

In [88]:
tf.layers.conv2d(inputs=_x, filters=32, kernel_size=(1,4), strides=1, padding="SAME")

<tf.Tensor 'conv2d_10/BiasAdd:0' shape=(?, 1, 100, 32) dtype=float32>

In [90]:
h = tf.layers.conv1d(inputs=x, filters=32, kernel_size=(3,), strides=1, padding="SAME")

In [91]:
h

<tf.Tensor 'conv1d_144/BiasAdd:0' shape=(?, 100, 32) dtype=float32>

In [93]:
h_2 = tf.layers.conv1d(inputs=h, filters=32, kernel_size=(3,), strides=1, padding="SAME")

In [94]:
h_2

<tf.Tensor 'conv1d_145/BiasAdd:0' shape=(?, 100, 32) dtype=float32>

In [80]:
_out = tf.layers.max_pooling1d(h, pool_size=(98,), strides=1)

In [81]:
_out

<tf.Tensor 'max_pooling1d_9/Squeeze:0' shape=(?, 1, 32) dtype=float32>

In [86]:
tf.reshape(_out, shape=(-1, 32))

<tf.Tensor 'Reshape_8:0' shape=(?, 32) dtype=float32>

In [82]:
tf.layers.conv1d(inputs=x, filters=32, kernel_size=(1,), strides=1, padding="SAME", dilation_rate=(2,))

<tf.Tensor 'conv1d_127/BiasAdd:0' shape=(?, 100, 32) dtype=float32>

In [83]:
def build_conv_block(x, filter_per_lvl=16):
    
    x_1 = tf.layers.conv1d(
        activation=tf.nn.relu,
        inputs=x, filters=filter_per_lvl, 
        kernel_size=(1,), strides=1, padding="SAME")
    
    x_2 = tf.layers.conv1d(
        activation=tf.nn.relu,
        inputs=x, filters=filter_per_lvl,
        kernel_size=(2,), strides=1, padding="SAME")
    
    x_3 = tf.layers.conv1d(
        activation=tf.nn.relu,
        inputs=x, filters=filter_per_lvl,
        kernel_size=(3,), strides=1, padding="SAME")
    
    x_4 = tf.layers.conv1d(
        activation=tf.nn.relu,
        inputs=x, filters=filter_per_lvl,
        kernel_size=(4,), strides=1, padding="SAME")
    
#     dilation rate 
      # http://vladlen.info/papers/dilated-convolutions.pdf
      # https://arxiv.org/pdf/1709.00179.pdf
    
#     x_2_dil_2 = tf.layers.conv1d(
#         activation=tf.nn.relu,
#         inputs=x, filters=filter_per_lvl, kernel_size=(2,),
#         strides=1, padding="SAME", dilation_rate=(2,))
#     x_2_dil_3 = tf.layers.conv1d(
#         activation=tf.nn.relu,
#         inputs=x, filters=filter_per_lvl, kernel_size=(2,),
#         strides=1, padding="SAME", dilation_rate=(3,))
#     x_3_dil_2 = tf.layers.conv1d(
#         activation=tf.nn.relu,
#         inputs=x, filters=filter_per_lvl, kernel_size=(3,), strides=1,
#         padding="SAME", dilation_rate=(2,))
#     x_3_dil_3 = tf.layers.conv1d(
#         activation=tf.nn.relu,
#         inputs=x, filters=filter_per_lvl, kernel_size=(3,),
#         strides=1, padding="SAME", dilation_rate=(3,))
    
    return tf.concat(values=[
        x_1, x_2, x_3, x_4,
#         x_2_dil_2, x_2_dil_3,
#         x_3_dil_2, x_3_dil_3,
    ], axis=2)

In [84]:
conv_block_1_1 = build_conv_block(x, filter_per_lvl=16)

In [95]:
conv_block_1_1

<tf.Tensor 'concat_17:0' shape=(?, 100, 128) dtype=float32>

In [85]:
conv_block_1_2 = build_conv_block(conv_block_1_1, filter_per_lvl=16)

In [96]:
conv_block_1_2

<tf.Tensor 'concat_18:0' shape=(?, 100, 128) dtype=float32>

In [49]:
pool_1 = tf.layers.max_pooling1d(conv_block_1_2, pool_size=(3,), strides=2, padding='SAME')

In [97]:
pool_1

<tf.Tensor 'max_pooling1d_5/Squeeze:0' shape=(?, 50, 128) dtype=float32>

In [50]:
conv_block_2_1 = build_conv_block(pool_1, filter_per_lvl=32)

In [51]:
conv_block_2_2 = build_conv_block(conv_block_2_1, filter_per_lvl=32)

In [52]:
pool_2 = tf.layers.max_pooling1d(conv_block_2_2, pool_size=(3,), strides=2, padding='SAME')

In [53]:
pool_2

<tf.Tensor 'max_pooling1d_6/Squeeze:0' shape=(?, 25, 256) dtype=float32>

In [54]:
conv_block_3_1 = build_conv_block(pool_2, filter_per_lvl=64)
conv_block_3_2 = build_conv_block(conv_block_3_1, filter_per_lvl=64)

In [55]:
conv_block_3_2

<tf.Tensor 'concat_14:0' shape=(?, 25, 512) dtype=float32>

In [60]:
conv_block_3_3 = tf.layers.conv1d(conv_block_3_2, filters=512, kernel_size=(1,), strides=1, padding='SAME')

In [62]:
global_pool = tf.layers.max_pooling1d(conv_block_3_3, pool_size=25,strides=1, padding='VALID') 

In [63]:
global_pool

<tf.Tensor 'max_pooling1d_7/Squeeze:0' shape=(?, 1, 512) dtype=float32>

In [64]:
dense = tf.reshape(global_pool, shape=(-1, 512))

In [65]:
dense

<tf.Tensor 'Reshape_3:0' shape=(?, 512) dtype=float32>

In [66]:
hidden = tf.layers.dense(dense, units=256, activation=tf.nn.relu)

In [68]:
hidden

<tf.Tensor 'dense/Relu:0' shape=(?, 256) dtype=float32>

In [69]:
output = tf.layers.dense(dense, units=4, activation=None)

In [70]:
output

<tf.Tensor 'dense_2/BiasAdd:0' shape=(?, 4) dtype=float32>

In [71]:
y = tf.placeholder(
    shape=[None, 4],
    dtype=tf.float32,
)

softmax = tf.nn.softmax(output)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=y))

In [72]:
y

<tf.Tensor 'Placeholder_1:0' shape=(?, 4) dtype=float32>

In [73]:
softmax

<tf.Tensor 'Softmax:0' shape=(?, 4) dtype=float32>

In [74]:
loss

<tf.Tensor 'Mean:0' shape=() dtype=float32>